In [257]:
import requests
import pandas as pd
from collections import defaultdict
from itertools import combinations
import json
import numpy as np

# Replace these with your repository details
owner = 'lucamenotti'
repo = '5v5s'
branch = 'main'


api_url = f'https://api.github.com/repos/{owner}/{repo}/contents'

# # Make a request to the GitHub API
# # headers = {'Authorization': f'token {token}'}
# response = requests.get(url)
# if response.status_code == 200:
#     contents = response.json()
#     # Filter out the JSON files and get their raw URLs
#     json_urls = [file['download_url'] for file in contents if file['name'].endswith('.json')]
# else:
#     print(f"Failed to fetch files: {response.status_code}")
#     json_urls = []


def fetch_files_from_github(api_url, path=''):
    """Fetches JSON files from the given GitHub repository path."""
    files = []
    response = requests.get(f'{api_url}/{path}')
    if response.status_code == 200:
        for item in response.json():
            if item['type'] == 'file' and item['name'].endswith('.json'):
                files.append(item['download_url'])
            elif item['type'] == 'dir':
                files.extend(fetch_files_from_github(api_url, item['path']))
    else:
        print(f"Failed to fetch files: {response.status_code} - {response.text}")
    return files

def fetch_json_from_url(url):
    """Fetches JSON data from the given URL."""
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch JSON from {url}: {response.status_code} - {response.text}")
    return None

# Fetch JSON files from the GitHub repository
json_urls = fetch_files_from_github(api_url)


# Read the JSON files into DataFrames
dataframes = [pd.read_json(url) for url in json_urls]

print(json_urls)

# Combine the DataFrames into a single DataFrame (if necessary)
combined_df = pd.concat(dataframes, ignore_index=True)


['https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/3470547662.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/3717449648.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4146140154.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4507145109.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4523056888.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4523110972.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4523153945.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4526839461.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4526868306.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4576421436.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4576450279.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/m

In [258]:
hold = []
for i in combined_df['participants']:
    hold.append(pd.Series(i))

df = combined_df.merge(pd.concat(hold, axis = 1).T, left_index= True, right_index=True,how = 'inner').drop(columns= ['participants', 'TOTAL_PLAYER_SCORE','TOTAL_SCORE_RANK','OBJECTIVE_PLAYER_SCORE','PUUID']).fillna(0)

In [259]:
player_aliases = {
    'EnderDragon7474': 'Ashalo1',
    'RaVe B00ST': 'Bryant',
    'CoomerHashira': 'Bryant',
    'Item Diversity': 'BloodStainedRose',
    'BigPeTe1239': 'DookieDemon12'
    # Add more mappings as needed
}
df['NAME'] = df['NAME'].apply(lambda x:player_aliases.get(x,x))

In [260]:
num_games = df[['matchId','NAME']].groupby('NAME').count()
num_games = num_games[num_games['matchId'] >= 10]
name_g10 = num_games.reset_index()['NAME']

df = df[df['NAME'].isin(np.array(name_g10))]

In [261]:
multi_type = []
for col in df.columns:
    if df[col].apply(type).nunique() != 1:
        multi_type.append(col)
multi_type

['ALL_IN_PINGS',
 'ASSIST_ME_PINGS',
 'BAIT_PINGS',
 'BARRACKS_TAKEDOWNS',
 'BASIC_PINGS',
 'COMMAND_PINGS',
 'DANGER_PINGS',
 'ENEMY_MISSING_PINGS',
 'ENEMY_VISION_PINGS',
 'GET_BACK_PINGS',
 'HOLD_PINGS',
 'HORDE_KILLS',
 'HQ_TAKEDOWNS',
 'ID',
 'LARGEST_ABILITY_DAMAGE',
 'LARGEST_ATTACK_DAMAGE',
 'LAST_TAKEDOWN_TIME',
 'Missions_ChampionsKilled',
 'Missions_CreepScore',
 'Missions_GoldFromStructuresDestroyed',
 'Missions_GoldFromTurretPlatesTaken',
 'Missions_HealingFromLevelObjects',
 'Missions_MinionsKilled',
 'Missions_TurretPlatesDestroyed',
 'NEED_VISION_PINGS',
 'ON_MY_WAY_PINGS',
 'PLAYER_AUGMENT_1',
 'PLAYER_AUGMENT_2',
 'PLAYER_AUGMENT_3',
 'PLAYER_AUGMENT_4',
 'PLAYER_SUBTEAM',
 'PLAYER_SUBTEAM_PLACEMENT',
 'PUSH_PINGS',
 'RETREAT_PINGS',
 'RIFT_HERALD_KILLS',
 'TOTAL_TIME_CROWD_CONTROL_DEALT_TO_CHAMPIONS',
 'TURRET_TAKEDOWNS',
 'VISION_CLEARED_PINGS',
 'WAS_LEAVER',
 'WAS_SURRENDER_DUE_TO_AFK']

In [262]:
for i in df.columns:
    df[i] = pd.to_numeric(df[i], errors = 'ignore')


In [263]:
multi_type = []
for col in df.columns:
    if df[col].apply(type).nunique() != 1:
        multi_type.append(col)
multi_type

[]

In [264]:
df.groupby('NAME').sum()['CHAMPIONS_KILLED'].sort_values(ascending = False)

NAME
adostraa           427
TylerToTheT212     410
IbelieveMeYou      364
Ashalo1            307
nigeriacoc         297
Dyneliex           144
DonutDude17        133
GasterBat          129
ihyash             116
Bryant              99
Babar               74
Little Bro Eman     74
MilkLL              63
DookieDemon12       55
uWumnati            47
batlife10           31
Name: CHAMPIONS_KILLED, dtype: int64

In [265]:
temp = df.groupby('matchId').sum()[['NAME']]
temp[temp['NAME'].str.contains('adostraa') != True].index

df[(df['matchId'] == 4939492680) | (df['matchId'] == 4939565193)]['NAME']

501     IbelieveMeYou
502            Bryant
503         batlife10
504     DookieDemon12
505           Ashalo1
506            ihyash
507       DonutDude17
509        nigeriacoc
511          uWumnati
512    TylerToTheT212
513            ihyash
514            Bryant
515           Ashalo1
517       DonutDude17
518     IbelieveMeYou
519        nigeriacoc
Name: NAME, dtype: object

In [266]:
df.groupby('NAME').count()['CHAMPIONS_KILLED'].sort_values(ascending = False)

NAME
adostraa           60
nigeriacoc         59
TylerToTheT212     56
IbelieveMeYou      54
Ashalo1            48
ihyash             39
DonutDude17        38
GasterBat          32
Bryant             30
uWumnati           27
batlife10          25
Dyneliex           23
DookieDemon12      21
Babar              19
MilkLL             13
Little Bro Eman    10
Name: CHAMPIONS_KILLED, dtype: int64

In [267]:
def standardize (series):
    return (series - series.mean())/series.std()

temp_df = df
standardize(temp_df[['NAME','TOTAL_DAMAGE_DEALT']].groupby('NAME').mean()).sort_values('TOTAL_DAMAGE_DEALT', ascending = False)


,TOTAL_DAMAGE_DEALT
NAME,
Little Bro Eman,1.451867
adostraa,1.409996
Dyneliex,0.703527
Ashalo1,0.605084
IbelieveMeYou,0.590569
MilkLL,0.504692
GasterBat,0.355304
TylerToTheT212,0.329152
Babar,0.241042


In [268]:
grouped = df.groupby(['matchId', 'TEAM'])
teammate_counts = defaultdict(lambda: defaultdict(int))

for (match_id, team_number), group in grouped:
    players = group['NAME'].tolist()
    # Create all possible pairs of teammates
    for player1, player2 in combinations(players, 2):
        # Increment the count for each pair (both ways, since it's undirected)
        teammate_counts[player1][player2] += 1
        teammate_counts[player2][player1] += 1

# Convert the result to a more readable format (DataFrame)
teammate_stats = []
for player, teammates in teammate_counts.items():
    for teammate, count in teammates.items():
        teammate_stats.append({'player_name': player, 'teammate_name': teammate, 'count': count})

teammate_df = pd.DataFrame(teammate_stats)

In [289]:
name = 'TylerToTheT212'

harb = teammate_df[teammate_df['player_name'] == name].sort_values('count',ascending = False)

win_change = {
    'Win' : True,
    'Fail' : False
}

copy_df = df.copy()[['NAME','WIN']]

copy_df['WIN'] = copy_df['WIN'].map(win_change)


c = copy_df.groupby('NAME').mean().reset_index()

c = c[c['NAME'].isin(np.array(harb['teammate_name']))]

harb = pd.merge(harb,c, left_on = 'teammate_name', right_on='NAME')

harb.drop(columns =['NAME'])
harb['WITH_HARB'] = (harb['WIN'] * harb['count'])
num = (harb['WIN'] * harb['count']).sum()/harb['count'].sum()
print(num)
harb

0.5152248655877678


,player_name,teammate_name,count,NAME,WIN,WITH_HARB
0,TylerToTheT212,adostraa,25,adostraa,0.59322,14.830508
1,TylerToTheT212,ihyash,24,ihyash,0.538462,12.923077
2,TylerToTheT212,GasterBat,23,GasterBat,0.6875,15.8125
3,TylerToTheT212,nigeriacoc,22,nigeriacoc,0.525424,11.559322
4,TylerToTheT212,Ashalo1,20,Ashalo1,0.395833,7.916667
5,TylerToTheT212,DonutDude17,16,DonutDude17,0.631579,10.105263
6,TylerToTheT212,IbelieveMeYou,14,IbelieveMeYou,0.481481,6.740741
7,TylerToTheT212,Bryant,12,Bryant,0.666667,8.0
8,TylerToTheT212,batlife10,12,batlife10,0.36,4.32
9,TylerToTheT212,DookieDemon12,10,DookieDemon12,0.333333,3.333333


In [294]:
role_df = df[['INDIVIDUAL_POSITION','NAME','matchId']].groupby(['NAME','INDIVIDUAL_POSITION']).count().reset_index()
role_df.sort_values('matchId',ascending=False).head(10)
role_df[role_df['NAME'] == 'uWumnati']

,NAME,INDIVIDUAL_POSITION,matchId
63,uWumnati,JUNGLE,1
64,uWumnati,MIDDLE,8
65,uWumnati,TOP,7
66,uWumnati,UTILITY,11


In [324]:
name = 'IbelieveMeYou'
champion = 'Poppy'

champ = df[['SKIN','NAME','WIN']]
champ['WIN'] = champ['WIN'].map(win_change)
champ = champ.groupby(['NAME','SKIN']).mean()
champ = champ


please = pd.merge(champ, df[['SKIN','NAME','matchId']].groupby(['NAME','SKIN']).count(), left_index=True,right_index= True)



please = please.reset_index()
please
please[please['NAME'] == name]
# please.reset_index()

C:\Users\zktho\AppData\Local\Temp\ipykernel_29000\2357869941.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  champ['WIN'] = champ['WIN'].map(win_change)


,NAME,SKIN,WIN,matchId
130,IbelieveMeYou,Ashe,1.0,2
131,IbelieveMeYou,Belveth,0.0,2
132,IbelieveMeYou,Caitlyn,0.0,2
133,IbelieveMeYou,Chogath,0.0,1
134,IbelieveMeYou,Diana,0.666667,3
135,IbelieveMeYou,DrMundo,1.0,1
136,IbelieveMeYou,Ekko,1.0,1
137,IbelieveMeYou,FiddleSticks,0.0,2
138,IbelieveMeYou,JarvanIV,0.0,1
139,IbelieveMeYou,Jhin,1.0,1


In [270]:
for col in df.columns:
    print(col)

matchId
gameDuration
gameVersion
ALL_IN_PINGS
ASSIST_ME_PINGS
ASSISTS
BAIT_PINGS
BARON_KILLS
BARRACKS_KILLED
BARRACKS_TAKEDOWNS
BASIC_PINGS
BOUNTY_LEVEL
CHAMPION_MISSION_STAT_0
CHAMPION_MISSION_STAT_1
CHAMPION_MISSION_STAT_2
CHAMPION_MISSION_STAT_3
CHAMPIONS_KILLED
CHAMPION_TRANSFORM
COMMAND_PINGS
CONSUMABLES_PURCHASED
DANGER_PINGS
DOUBLE_KILLS
DRAGON_KILLS
ENEMY_MISSING_PINGS
ENEMY_VISION_PINGS
EXP
FRIENDLY_DAMPEN_LOST
FRIENDLY_HQ_LOST
FRIENDLY_TURRET_LOST
GAME_ENDED_IN_EARLY_SURRENDER
GAME_ENDED_IN_SURRENDER
GET_BACK_PINGS
GOLD_EARNED
GOLD_SPENT
HOLD_PINGS
HORDE_KILLS
HQ_KILLED
HQ_TAKEDOWNS
ID
INDIVIDUAL_POSITION
ITEM0
ITEM1
ITEM2
ITEM3
ITEM4
ITEM5
ITEM6
ITEMS_PURCHASED
KEYSTONE_ID
KILLING_SPREES
LARGEST_ABILITY_DAMAGE
LARGEST_ATTACK_DAMAGE
LARGEST_CRITICAL_STRIKE
LARGEST_KILLING_SPREE
LARGEST_MULTI_KILL
LAST_TAKEDOWN_TIME
LEVEL
LONGEST_TIME_SPENT_LIVING
MAGIC_DAMAGE_DEALT_PLAYER
MAGIC_DAMAGE_DEALT_TO_CHAMPIONS
MAGIC_DAMAGE_TAKEN
MINIONS_KILLED
Missions_ChampionsKilled
Missions_Creep